In [37]:
import requests
import json

response_query_graph = {}
response_kg_edges = {}
response_kg_nodes = {}
response_results = []
response_aux_graphs = {}
import orjson

In [38]:
AC_URL = "https://answercoalesce.renci.org/query"

In [39]:
def generate_trapi_query(input_node_type,output_node_type,input_curie,predicate,input_is_subject=True):
    envelope = {"message":{"query_graph": {"nodes":{"input":{},"output":{}}, "edges":{"edge_0":{}}}}}
    input_node = envelope["message"]["query_graph"]["nodes"]["input"]
    input_node["categories"] = [input_node_type]
    input_node["ids"] = [input_curie]
    output_node = envelope["message"]["query_graph"]["nodes"]["output"]
    output_node["categories"] = [output_node_type]
    query_edge = envelope["message"]["query_graph"]["edges"]["edge_0"]
    if input_is_subject:
        query_edge["subject"] = "output"
        query_edge["object"] = "input"
    else:
        query_edge["subject"] = "input"
        query_edge["object"] = "output"
    query_edge["predicates"]  = [ predicate ]
    query_edge["knowledge_type"] = "inferred"
    return envelope

In [40]:
def disease_to_drugs(disease, params=None, print_query=False ):
    input_type = "biolink:Disease"
    output_type = "biolink:Drug"
    predicate = "biolink:treats"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, disease, predicate, input_is_subject)
    if params:
        query.update(params)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()


In [41]:
def disease_to_genes(disease, params=None, print_query=False ):
    input_type = "biolink:Disease"
    output_type = "biolink:Gene"
    predicate = "biolink:genetically_associated_with"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, disease, predicate, input_is_subject)
    if params:
        query.update(params)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()


In [42]:
def phenotype_to_genes(phenotype, params=None, print_query=False ):
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Gene"
    predicate = "biolink:affects"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, phenotype, predicate, input_is_subject)
    if params:
        query.update(params)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()


In [43]:
def gene_to_phenotypes(gene, params=None, print_query=False ):
    input_type = "biolink:Gene"
    output_type = "biolink:PhenotypicFeature"
    predicate = "biolink:has_phenotype"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, gene, predicate, input_is_subject)
    if params:
        query.update(params)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()


In [44]:
def disease_to_phenotype(disease, params=None, print_query=False ):
    input_type = "biolink:Disease"
    output_type = "biolink:PhenotypicFeature"
    predicate = "biolink:has_phenotype"
    input_is_subject = "True"
    query = generate_trapi_query( input_type, output_type, disease, predicate, input_is_subject)
    if params:
        query.update(params)
    if print_query:
        print(json.dumps(query,indent=2))
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()


In [45]:
def print_results(resp):
    global response_query_graph, response_aux_graphs, response_results, response_kg_edges, response_kg_nodes 
    response_query_graph = resp["message"]["query_graph"]
    response_aux_graphs = resp["message"]["auxiliary_graphs"]
    response_results = resp["message"]["results"]
    response_kg_edges = resp["message"]["knowledge_graph"]["edges"]
    response_kg_nodes = resp["message"]["knowledge_graph"]["nodes"]
    
    for i, result in enumerate(resp["message"]["results"]):
        nb = result["node_bindings"]["output"][0]["id"]
        name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
        print(f"{i} | {nb} | {name}") 

In [46]:
# Now let's explore the path for any of the results
def print_result_support_graphs(i, answerset):
    #Let's see the inferred_edge in the KG then extract it support graphs
    for eid, edge in response_results[i]["analyses"][0]["edge_bindings"].items():
        inferred_edge = edge[0]["id"]
        support_graphs = [attributes["value"] for attributes in response_kg_edges[inferred_edge]["attributes"] if attributes["attribute_type_id"] == "biolink:support_graphs"]
        print(f'{inferred_edge} has {len(support_graphs)} evidential Paths')
        print("=============")
        for i, sg in enumerate(support_graphs):
            print(f"{i} | {sg}")
        print()
        print("**e_Inferred.....: graph/edge enriched paths")
        print("**n_Inferred.....: node/property enriched paths")
        return support_graphs

In [47]:
def explore_one_support_graph(i, support_graphs):
    # Tracing the support graph to the auxiliary components
    aux_graph_edges = response_aux_graphs.get(support_graphs[i]).get("edges")
    print(f'{support_graphs[i]} has ---> {len(aux_graph_edges)} edges')
    print("=============")
    for i, aux_edge in enumerate(aux_graph_edges):
        print(i, aux_edge)
    
    print()
    print("**Each Inferred edge support graph's aux_graph has 3 edges**")
    print("- Inferred_node to the enrichment")
    print("- Enrichment to the Groupset (lookup_list)")
    print("- The Groupset to the qg Curie")
    
    inf2enrichment_aux_graph_edge = ''
    enrich2group_aux_graph_edge = ''
    group2curie_aux_graph_edge = ''
    for aedge in aux_graph_edges:
        support = [attributes["value"] for attributes in response_kg_edges[aedge]["attributes"]]
        if support:
            if isinstance(support[0], list):
                enrich2group_aux_graph_edge = aedge
            else:
                group2curie_aux_graph_edge = aedge
        else:
            inf2enrichment_aux_graph_edge = aedge
    return inf2enrichment_aux_graph_edge, enrich2group_aux_graph_edge, group2curie_aux_graph_edge

In [48]:
def explore_one_support_graphs_first_edge(inf2enrichment_aux_graph_edge):
    # The first edge: 
    inf2enrichmentedge = response_kg_edges[inf2enrichment_aux_graph_edge]
    
    print(f"{response_kg_nodes[inf2enrichmentedge['subject']]['name']} -({inf2enrichmentedge['predicate']})-> {response_kg_nodes[inf2enrichmentedge['object']]['name']}")
    print()
    print(f"The first one: {inf2enrichmentedge}")
    
    print()
    print("***inferred_node -(rel)- enrichment usually has no support graph so we stop digging***")

In [49]:
def explore_one_support_graphs_second_edge(enrich2group_aux_graph_edge):
    print(" This is the enrichment - (rel) - lookup_list. The Major AnswerCoalesce Module. It has many support graph, so we dig further ")
    print("=========")
    enrichment2group_edge = response_kg_edges[enrich2group_aux_graph_edge]
    print(enrichment2group_edge)
    print()
    print(f"{response_kg_nodes[enrichment2group_edge['subject']]['name']} -({enrichment2group_edge['predicate']})-> {response_kg_nodes[enrichment2group_edge['object']]['name']}")
    print()
    for attributes in enrichment2group_edge["attributes"]:
        enrichment2group_support_graphs = attributes["value"]
        # Each of these exists in the auxiliary graph
        for i, e2group_sp in enumerate(enrichment2group_support_graphs):
            e2group_edges = response_aux_graphs[e2group_sp]["edges"]
            theedges0 = response_kg_edges[e2group_edges[0]]
            theedges1 = response_kg_edges[e2group_edges[1]]
            if "qualifiers" in theedges0:
                qc = '_'.join([q["qualifier_value"] for q in theedges0["qualifiers"]])
                predicate0 = f"{theedges0["predicate"]}__{qc}"
            else:
                predicate0 = theedges0["predicate"]
                
            if "qualifiers" in theedges1:
                qc = '_'.join([q["qualifier_value"] for q in theedges1["qualifiers"]])
                predicate1 = f"{theedges1["predicate"]}__{qc}"
            else:
                predicate1 = theedges1["predicate"]
            edge0pval = [attr["value"] for attr in theedges0["attributes"] if attr["attribute_type_id"] == "biolink:p_value"]
            edge1pval = [attr["value"] for attr in theedges1["attributes"] if attr["attribute_type_id"] == "biolink:p_value"]
            if edge0pval:
                pvalue = edge0pval[0]
            else:
                pvalue = edge1pval[0]
            print(f'Path{i}: {response_kg_nodes[theedges0["subject"]]["name"]}--({predicate0})-->{response_kg_nodes[theedges0["object"]]["name"]} ^ {response_kg_nodes[theedges1["subject"]]["name"]}--({predicate1})-->{response_kg_nodes[theedges1["object"]]["name"]} | {pvalue}')
            print()
# 4 members of set uuid affects `NCBIGene:83817`         

In [50]:
def explore_one_support_graphs_third_edge(group2curie_aux_graph_edge):
    print("The Groupset- qg Curie edge has support graph of members of the group, so we stop digging")
    print("=========")
    
    print(response_kg_edges[group2curie_aux_graph_edge])
    print()
    print(f"{group2curie_aux_graph_edge}:===>")
    for attributes in response_kg_edges[group2curie_aux_graph_edge]["attributes"]:
        sgs = attributes['value']
        sgedges = sorted(response_aux_graphs[sgs]["edges"])
        j = 0; path = 1
        while j < len(sgedges)-1:
            p1 = response_kg_edges[sgedges[j]]
            p2 = response_kg_edges[sgedges[j+1]]
            print(f'Path{path}: {response_kg_nodes[p2["subject"]]["name"]}-({p2["predicate"]})->{response_kg_nodes[p2["object"]]["name"]} ^ {response_kg_nodes[p1["subject"]]["name"]}-({p1["predicate"]})->{response_kg_nodes[p1["object"]]["name"]}')
            j+=2
            path+=1


In [51]:
disease = "MONDO:0004975" #Alzheimers
p_value = 1e-5; result_length = 100; predicates_to_exclude = [ "biolink:causes", "biolink:biomarker_for", "biolink:contraindicated_for", "biolink:contraindicated_in", "biolink:contributes_to", "biolink:has_adverse_event", "biolink:causes_adverse_event", "biolink:treats_or_applied_or_studied_to_treat" ]
params = {"parameters": {"pvalue_threshold": p_value, "result_length": result_length, "predicates_to_exclude": predicates_to_exclude}}

disease_to_drug_response = disease_to_drugs(disease, params, print_query=False)
print_results(disease_to_drug_response)

200
0 | CHEBI:27953 | physostigmine
1 | CHEBI:553827 | bambuterol
2 | CHEBI:45713 | trans-resveratrol
3 | CHEBI:8711 | quinacrine
4 | CHEBI:31981 | periciazine
5 | CHEBI:3547 | cephradine
6 | CHEBI:82866 | sertaconazole
7 | CHEBI:4754 | econazole
8 | CHEBI:134709 | pitolisant
9 | CHEBI:3510 | ceftibuten
10 | CHEBI:7565 | nifedipine
11 | CHEBI:15854 | quinine
12 | CHEBI:27446 | phenoxymethylpenicillin
13 | CHEBI:16523 | D-serine
14 | CHEBI:474053 | cefazolin
15 | CHEBI:7514 | neostigmine
16 | CHEBI:49575 | diazepam
17 | CHEBI:5775 | hydralazine
18 | CHEBI:9150 | simvastatin
19 | CHEBI:3480 | cefamandole
20 | CHEBI:6923 | miconazole
21 | CHEBI:9711 | triflupromazine
22 | CHEBI:27607 | thymol
23 | CHEBI:40303 | lovastatin
24 | CHEBI:93248 | (6R,7R)-7-[[2-(2-amino-4-thiazolyl)-2-(carboxymethoxyimino)-1-oxoethyl]amino]-3-ethenyl-8-oxo-5-thia-1-azabicyclo[4.2.0]oct-2-ene-2-carboxylic acid
25 | CHEBI:3611 | chlordiazepoxide
26 | CHEBI:69478 | cannabidiol
27 | CHEBI:15354 | choline
28 | CHEBI:

# Let's trace one result

In [52]:
index = 1
support_graphs = print_result_support_graphs(index, disease_to_drug_response)

CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975 has 7 evidential Paths
0 | n_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:has_chemical_role_CHEBI_ROLE_neurotransmitter_agent
1 | e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:590
2 | e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_UniProtKB:P81908
3 | n_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:has_chemical_role_CHEBI_ROLE_EC_3.1.1_carboxylic_ester_hydrolase_inhibitor
4 | e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:65036
5 | e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:83817
6 | e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:4

In [53]:
index = 1
inf2enrichment_aux_graph_edge, enrich2group_aux_graph_edge, group2curie_aux_graph_edge = explore_one_support_graph(index, support_graphs)

e_Inferred_SG:_CHEBI:553827_Inferred_to_biolink:treats_MONDO:0004975_via_CHEBI:553827_biolink:affects_NCBIGene:590 has ---> 3 edges
0 uuid:1_biolink:treats_MONDO:0004975
1 CHEBI:553827_biolink:affects_NCBIGene:590
2 e_uuid:1_biolink:affects_NCBIGene:590

**Each Inferred edge support graph's aux_graph has 3 edges**
- Inferred_node to the enrichment
- Enrichment to the Groupset (lookup_list)
- The Groupset to the qg Curie


In [54]:
explore_one_support_graphs_first_edge(inf2enrichment_aux_graph_edge)

bambuterol -(biolink:affects)-> BCHE

The first one: {'subject': 'CHEBI:553827', 'object': 'NCBIGene:590', 'predicate': 'biolink:affects', 'sources': [{'resource_id': 'infores:pharos', 'resource_role': 'primary_knowledge_source'}], 'qualifiers': [{'qualifier_type_id': 'biolink:object_direction_qualifier', 'qualifier_value': 'decreased'}, {'qualifier_type_id': 'biolink:object_aspect_qualifier', 'qualifier_value': 'activity'}], 'attributes': []}

***inferred_node -(rel)- enrichment usually has no support graph so we stop digging***


In [55]:
explore_one_support_graphs_second_edge(enrich2group_aux_graph_edge)

 This is the enrichment - (rel) - lookup_list. The Major AnswerCoalesce Module. It has many support graph, so we dig further 
{'subject': 'uuid:1', 'object': 'NCBIGene:590', 'predicate': 'biolink:affects', 'sources': [{'resource_id': 'infores:answercoalesce', 'resource_role': 'primary_knowledge_source'}], 'qualifiers': [], 'attributes': [{'attribute_type_id': 'biolink:support_graphs', 'value': ['SG:_e_CHEBI:42944_biolink:affects_NCBIGene:590', 'SG:_e_CHEBI:45980_biolink:affects_NCBIGene:590', 'SG:_e_CHEBI:8874_biolink:affects_NCBIGene:590', 'SG:_e_CHEBI:53289_biolink:affects_NCBIGene:590', 'SG:_e_CHEBI:15355_biolink:affects_NCBIGene:590'], 'attribute_source': 'infores:answercoalesce'}]}

uuid:1 -(biolink:affects)-> BCHE

Path0: galanthamine--(biolink:member_of)-->uuid:1 ^ galanthamine--(biolink:affects__abundance)-->BCHE | 4.775747956552919e-06

Path1: tacrine--(biolink:affects__abundance)-->BCHE ^ tacrine--(biolink:member_of)-->uuid:1 | 4.775747956552919e-06

Path2: rivastigmine--(bio

In [56]:
explore_one_support_graphs_third_edge(group2curie_aux_graph_edge)

The Groupset- qg Curie edge has support graph of members of the group, so we stop digging
{'subject': 'uuid:1', 'object': 'MONDO:0004975', 'predicate': 'biolink:treats', 'sources': [{'resource_id': 'infores:answercoalesce', 'resource_role': 'primary_knowledge_source'}], 'attributes': [{'attribute_type_id': 'biolink:support_graphs', 'value': 'SG:_uuid:1_biolink:treats_MONDO:0004975', 'attribute_source': 'infores:answercoalesce'}]}

uuid:1_biolink:treats_MONDO:0004975:===>
Path1: acetylcholine-(biolink:member_of)->uuid:1 ^ acetylcholine-(biolink:treats)->Alzheimer disease
Path2: benzatropine-(biolink:member_of)->uuid:1 ^ benzatropine-(biolink:treats)->Alzheimer disease
Path3: galanthamine-(biolink:member_of)->uuid:1 ^ galanthamine-(biolink:treats)->Alzheimer disease
Path4: tacrine-(biolink:member_of)->uuid:1 ^ tacrine-(biolink:treats)->Alzheimer disease
Path5: donepezil-(biolink:member_of)->uuid:1 ^ donepezil-(biolink:treats)->Alzheimer disease
Path6: haloperidol-(biolink:member_of)->uui

### disease_to_gene

In [57]:
disease = "DOID:0050430" # multiple endocrine neoplasia type 2A 
params = {"parameters": {"pvalue_threshold": 1e-5, "result_length": 1000, "predicates_to_exclude": []}}
disease_to_gene_response = disease_to_genes(disease, params, print_query=False)
print_results(disease_to_gene_response)

200
0 | NCBIGene:9180 | OSMR
1 | NCBIGene:348 | APOE
2 | NCBIGene:284217 | LAMA1
3 | NCBIGene:133396 | IL31RA
4 | NCBIGene:325 | APCS
5 | NCBIGene:2261 | FGFR3
6 | NCBIGene:5290 | PIK3CA
7 | NCBIGene:4128 | MAOA
8 | NCBIGene:23095 | KIF1B
9 | NCBIGene:1644 | DDC
10 | NCBIGene:2668 | GDNF
11 | NCBIGene:7428 | VHL
12 | NCBIGene:1113 | CHGA
13 | NCBIGene:4129 | MAOB
14 | NCBIGene:4149 | MAX
15 | NCBIGene:1312 | COMT
16 | NCBIGene:7054 | TH
17 | NCBIGene:1621 | DBH
18 | NCBIGene:4549 | RNR1
19 | NCBIGene:3440 | IFNA2
20 | NCBIGene:6752 | SSTR2
21 | NCBIGene:54949 | SDHAF2
22 | NCBIGene:6530 | SLC6A2
23 | NCBIGene:6389 | SDHA
24 | NCBIGene:6391 | SDHC
25 | NCBIGene:8314 | BAP1
26 | NCBIGene:133 | ADM
27 | NCBIGene:152 | ADRA2C
28 | NCBIGene:4803 | NGF
29 | NCBIGene:146 | ADRA1D
30 | NCBIGene:6855 | SYP
31 | NCBIGene:148 | ADRA1A
32 | NCBIGene:151 | ADRA2B
33 | NCBIGene:150 | ADRA2A
34 | NCBIGene:147 | ADRA1B
35 | NCBIGene:2641 | GCG
36 | NCBIGene:1543 | CYP1A1
37 | NCBIGene:5741 | PTH
38 | 

In [58]:
# Let's trace one result
index = 1
support_graphs = print_result_support_graphs(index, disease_to_drug_response)

NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430 has 8 evidential Paths
0 | e_Inferred_SG:_NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430_via_NCBIGene:348_biolink:genetically_associated_with_MONDO:0003634
1 | e_Inferred_SG:_NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430_via_NCBIGene:348_biolink:genetically_associated_with_MONDO:0015277
2 | e_Inferred_SG:_NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430_via_NCBIGene:348_biolink:genetically_associated_with_MONDO:0008903
3 | e_Inferred_SG:_NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430_via_NCBIGene:348_biolink:genetically_associated_with_MONDO:0005027
4 | e_Inferred_SG:_NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430_via_NCBIGene:348_biolink:genetically_associated_with_MONDO:0005365
5 | e_Inferred_SG:_NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430_via_NCBIGe

In [59]:
index = 1
inf2enrichment_aux_graph_edge, enrich2group_aux_graph_edge, group2curie_aux_graph_edge = explore_one_support_graph(
    index, support_graphs)

e_Inferred_SG:_NCBIGene:348_Inferred_to_biolink:genetically_associated_with_DOID:0050430_via_NCBIGene:348_biolink:genetically_associated_with_MONDO:0015277 has ---> 3 edges
0 uuid:1_biolink:genetically_associated_with_DOID:0050430
1 e_MONDO:0015277_biolink:genetically_associated_with_uuid:1
2 NCBIGene:348_biolink:genetically_associated_with_MONDO:0015277

**Each Inferred edge support graph's aux_graph has 3 edges**
- Inferred_node to the enrichment
- Enrichment to the Groupset (lookup_list)
- The Groupset to the qg Curie


In [60]:
explore_one_support_graphs_first_edge(inf2enrichment_aux_graph_edge)
explore_one_support_graphs_second_edge(enrich2group_aux_graph_edge)
explore_one_support_graphs_third_edge(group2curie_aux_graph_edge)


APOE -(biolink:genetically_associated_with)-> medullary thyroid gland carcinoma

The first one: {'subject': 'NCBIGene:348', 'object': 'MONDO:0015277', 'predicate': 'biolink:genetically_associated_with', 'sources': [{'resource_id': 'infores:diseases', 'resource_role': 'primary_knowledge_source'}, {'resource_id': 'infores:pharos', 'resource_role': 'aggregator_knowledge_source'}], 'attributes': []}

***inferred_node -(rel)- enrichment usually has no support graph so we stop digging***
 This is the enrichment - (rel) - lookup_list. The Major AnswerCoalesce Module. It has many support graph, so we dig further 
{'subject': 'MONDO:0015277', 'object': 'uuid:1', 'predicate': 'biolink:genetically_associated_with', 'sources': [{'resource_id': 'infores:answercoalesce', 'resource_role': 'primary_knowledge_source'}], 'qualifiers': [], 'attributes': [{'attribute_type_id': 'biolink:support_graphs', 'value': ['SG:_e_MONDO:0015277_biolink:genetically_associated_with_NCBIGene:796', 'SG:_e_MONDO:0015277_b

# phenotype_to_gene

In [61]:
phenotype = "HP:0003637" # Reduced circulating 4-Hydroxyphenylpyruvate dioxygenase activity
params = {"parameters": {"pvalue_threshold": 1e-3, "result_length": 1000, "predicates_to_exclude": []}}
phenotype_to_gene_response = phenotype_to_genes(phenotype, params, print_query=False)
print_results(phenotype_to_gene_response)

200
0 | NCBIGene:25400 | Camk2a
1 | NCBIGene:231 | AKR1B1
2 | NCBIGene:29531 | Hpd
3 | NCBIGene:1488 | CTBP2
4 | NCBIGene:760 | CA2
5 | NCBIGene:71934 | Car13
6 | NCBIGene:5970 | RELA
7 | NCBIGene:54578 | UGT1A6
8 | NCBIGene:5743 | PTGS2
9 | NCBIGene:9429 | ABCG2
10 | NCBIGene:10864 | SLC22A7
11 | NCBIGene:241 | ALOX5AP
12 | NCBIGene:3172 | HNF4A
13 | NCBIGene:9356 | SLC22A6
14 | NCBIGene:762 | CA4
15 | NCBIGene:840 | CASP7
16 | NCBIGene:5310 | PKD1
17 | NCBIGene:2936 | GSR
18 | NCBIGene:5594 | MAPK1
19 | NCBIGene:1571 | CYP2E1
20 | NCBIGene:10728 | PTGES3
21 | NCBIGene:761 | CA3
22 | NCBIGene:5742 | PTGS1
23 | NCBIGene:5184 | PEPD
24 | NCBIGene:1576 | CYP3A4
25 | NCBIGene:766 | CA7
26 | NCBIGene:759 | CA1
27 | NCBIGene:6647 | SOD1
28 | NCBIGene:59341 | TRPV4
29 | NCBIGene:51 | ACOX1
30 | NCBIGene:2876 | GPX1
31 | NCBIGene:6777 | STAT5B
32 | NCBIGene:6648 | SOD2
33 | NCBIGene:771 | CA12
34 | NCBIGene:768 | CA9
35 | NCBIGene:10587 | TXNRD2
36 | NCBIGene:3356 | HTR2A
37 | NCBIGene:1152 |

# gene_to_phenotype

In [64]:
gene = "NCBIGene:122481" #"AK7"
params = {"parameters": {"pvalue_threshold": 1e-5, "result_length": 100, "predicates_to_exclude": []}}
gene_to_phenotype_response = gene_to_phenotypes(gene, params=params, print_query=False )
print_results(gene_to_phenotype_response)

200
0 | HP:0033393 | Irregularly shaped sperm tail
1 | HP:0032562 | Tapered sperm head
2 | HP:0034011 | Reduced progressive sperm motility
3 | HP:0034811 | Bent sperm flagella
4 | HP:0025437 | Macrozoospermia
5 | HP:0032561 | Microcephalic sperm head
6 | HP:0001288 | Gait disturbance
7 | HP:0002607 | Bowel incontinence
8 | HP:0000020 | Urinary incontinence
9 | HP:0001263 | Global developmental delay
10 | HP:0008669 | Abnormal spermatogenesis
11 | HP:0012867 | Abnormal sperm mid-piece morphology
12 | HP:0011961 | Non-obstructive azoospermia
13 | HP:0000837 | Increased circulating gonadotropin level
14 | HP:0008734 | Decreased testicular size
15 | HP:0011962 | Obstructive azoospermia
16 | HP:0003581 | Adult onset
17 | HP:0005425 | Recurrent sinopulmonary infections
18 | HP:0032016 | Abnormal sputum
19 | HP:0011463 | Childhood onset
20 | HP:0032543 | Lithoptysis
21 | HP:0005301 | Persistent left superior vena cava
22 | HP:0010772 | Anomalous pulmonary venous return
23 | HP:0025177 | Perib

# disease_to_phenotype

In [63]:
disease = "MONDO:0005147" # colonic neoplasm
params = {"parameters": {"pvalue_threshold": 1e-5, "result_length": 100, "predicates_to_exclude": []}}
disease_to_phenotype_response = disease_to_phenotype(disease, params=params, print_query=False )
print_results(disease_to_phenotype_response)

200
0 | HP:0002936 | Distal sensory impairment
1 | HP:0002355 | Difficulty walking
2 | HP:0002522 | Areflexia of lower limbs
3 | HP:0001761 | Pes cavus
4 | HP:0003393 | Thenar muscle atrophy
5 | HP:0011462 | Young adult onset
6 | HP:0003431 | Decreased motor nerve conduction velocity
7 | HP:0001765 | Hammertoe
8 | HP:0002460 | Distal muscle weakness
9 | HP:0011463 | Childhood onset
10 | HP:0003596 | Middle age onset
11 | HP:0001386 | Joint swelling
12 | HP:0001510 | Growth delay
13 | HP:0011227 | Elevated circulating C-reactive protein concentration
14 | HP:0001442 | Typified by somatic mosaicism
15 | HP:0002829 | Arthralgia
16 | HP:0000855 | Insulin resistance
17 | HP:0000969 | Edema
18 | HP:0002716 | Lymphadenopathy
19 | HP:0002027 | Abdominal pain
20 | HP:0002202 | Pleural effusion
21 | HP:0002240 | Hepatomegaly
22 | HP:0001945 | Fever
23 | HP:0001744 | Splenomegaly
24 | HP:0032154 | Aphthous ulcer
25 | HP:0031819 | Increased waist to hip ratio
26 | HP:0001034 | Hypermelanotic macul